# 1. Setup Mario

In [1]:
!pip install gym_super_mario_bros==7.3.0 nes_py

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for nes-py: filename=nes_py-8.1.8-cp38-cp38-win_amd64.whl size=48004 sha256=e1863b7dfbc8ea851c3b30120c2f2b67ccbed70a90b6149fd0793a1374a5a88d
  Stored in directory: c:\users\computing\appdata\local\pip\cache\wheels\8d\6e\f0\113c979eba40def28ee9b3c81a4adec00386106d81fb3bc2c2
  Created wheel for gym: filename=gym-0.23.1-py3-none-any.whl size=701358 sha256=e4fcaeab6fcffb4f13deafbd84abf2f29443000809abbcba611899d750e4743e
  Stored in directory: c:\users\computing\appdata\local\pip\cache\wheels\78\28\77\b0c74e80a2a4faae0161d5c53bc4f8e436e77aedc79136ee13
Successfully built nes-py gym
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-met

In [3]:
# Import the game
import gym_super_mario_bros
# Import the Joypad wrapper
from nes_py.wrappers import JoypadSpace
# Import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

C:\ProgramData\Anaconda3\lib\site-packages\gym\envs\registration.py:505: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3` with the environment ID `SuperMarioBros-v3`.
  logger.warn(


C:\ProgramData\Anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


# 2. Preprocess Environment

In [5]:
# Install pytorch
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
  Attempting uninstall: torch
    Found existing installation: torch 1.9.1
    Uninstalling torch-1.9.1:
      Successfully uninstalled torch-1.9.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.1
    Uninstalling torchvision-0.10.1:
      Successfully uninstalled torchvision-0.10.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.9.1
    Uninstalling torchaudio-0.9.1:
      Successfully uninstalled torchaudio-0.9.1


In [12]:
# Install stable baselines for RL stuff
!pip install stable-baselines3[extra]

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616828 sha256=693f5f5c2d631e11ce0ad24a0542566c1e047431f733369dc4df89067d9990f1
  Stored in directory: c:\users\computing\appdata\local\pip\cache\wheels\27\6d\b3\a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=446447 sha256=7ad45d2b5b6ee5a7d

In [13]:
# Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

# 3. Train the RL Model

In [19]:
# Import os for file path management
import os 
# Import PPO for algos
from stable_baselines3 import DQN
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [20]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [21]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [22]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [23]:
# This is the AI model started WITH DQN
model = DQN('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,buffer_size=150) 

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [24]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=2000000, callback=callback)

Logging to ./logs/DQN_1


C:\ProgramData\Anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.81e+04 |
|    ep_rew_mean      | 608      |
|    exploration_rate | 0.657    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 322      |
|    time_elapsed     | 224      |
|    total_timesteps  | 72253    |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.0305   |
|    n_updates        | 5563     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.39e+04 |
|    ep_rew_mean      | 910      |
|    exploration_rate | 0.472    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 232      |
|    time_elapsed     | 479      |
|    total_timesteps  | 111237   |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.0603   |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.54e+03 |
|    ep_rew_mean      | 1.46e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 68       |
|    fps              | 164      |
|    time_elapsed     | 1466     |
|    total_timesteps  | 240946   |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.383    |
|    n_updates        | 47736    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.4e+03  |
|    ep_rew_mean      | 1.45e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 72       |
|    fps              | 163      |
|    time_elapsed     | 1497     |
|    total_timesteps  | 245004   |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.351    |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.2e+03  |
|    ep_rew_mean      | 1.51e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 132      |
|    fps              | 154      |
|    time_elapsed     | 2076     |
|    total_timesteps  | 319860   |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.613    |
|    n_updates        | 67464    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.19e+03 |
|    ep_rew_mean      | 1.52e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 136      |
|    fps              | 153      |
|    time_elapsed     | 2116     |
|    total_timesteps  | 324983   |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.387    |
|    n_updates      

...

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.09e+03 |
|    ep_rew_mean      | 1.47e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 580      |
|    fps              | 134      |
|    time_elapsed     | 8860     |
|    total_timesteps  | 1191306  |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.572    |
|    n_updates        | 285326   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.06e+03 |
|    ep_rew_mean      | 1.46e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 584      |
|    fps              | 134      |
|    time_elapsed     | 8912     |
|    total_timesteps  | 1198003  |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.286    |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.89e+03 |
|    ep_rew_mean      | 1.69e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 133      |
|    time_elapsed     | 10270    |
|    total_timesteps  | 1370921  |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.987    |
|    n_updates        | 330230   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.86e+03 |
|    ep_rew_mean      | 1.71e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 133      |
|    time_elapsed     | 10359    |
|    total_timesteps  | 1382152  |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.569    |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.43e+03 |
|    ep_rew_mean      | 1.93e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 708      |
|    fps              | 132      |
|    time_elapsed     | 11419    |
|    total_timesteps  | 1517652  |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 1.1      |
|    n_updates        | 366912   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.39e+03 |
|    ep_rew_mean      | 1.93e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 712      |
|    fps              | 132      |
|    time_elapsed     | 11462    |
|    total_timesteps  | 1523241  |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.887    |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.39e+03 |
|    ep_rew_mean      | 1.9e+03  |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 772      |
|    fps              | 132      |
|    time_elapsed     | 12686    |
|    total_timesteps  | 1679565  |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.693    |
|    n_updates        | 407391   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.42e+03 |
|    ep_rew_mean      | 1.89e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 776      |
|    fps              | 132      |
|    time_elapsed     | 12776    |
|    total_timesteps  | 1690967  |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 1.27     |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.66e+03 |
|    ep_rew_mean      | 1.89e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 836      |
|    fps              | 131      |
|    time_elapsed     | 13967    |
|    total_timesteps  | 1841984  |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 1.39     |
|    n_updates        | 447995   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.64e+03 |
|    ep_rew_mean      | 1.89e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 840      |
|    fps              | 131      |
|    time_elapsed     | 14074    |
|    total_timesteps  | 1855601  |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.875    |
|    n_updates      

In [26]:
model.save('thisisatedqnmodel')

In [27]:
########################################################################################################

# 4. Test it Out

In [ ]:
# Load model
model = PPO.load('./train/best_model_1000000')

In [ ]:
state = env.reset()

In [ ]:
# Start the game 
state = env.reset()
# Loop through the game
while True: 
    
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()